In [54]:
import numpy as np
import matplotlib.pyplot as plt
import iop_ext as iop
import lf14 as lf
import math

In [198]:
#assigning constants

#use NEA & use constants regarding a singular exoplanet
au=1.49598e11 #meters - astronomical unit(dist. from Earth to Sun)
L_sun=3.828e26 #Watts - luminosity of our star
M_sun=1.989e30 #Kg - mass of our star
M_ea=5.97219e24 #Kg - mass of earth
R_ea=6.371e6 #meters - radius of earth
L_star=0.5*L_sun #placeholder 
M_star=0.665*M_sun
Teq=871 # Equilibrium temperature in Kelvin
a= 0.0422*au #semimajor axis
a_0 = 0.5 ##slope at which L_He decreases over time
yr=3600*24*365 #Converting year to seconds
t_sat=1e8*yr #100 million years to seconds
cmf_p=0.325 #reduced core mass fraction
G=6.67e-11 #Universal Gravitational Constant
L_sat=(10**-3.5)*L_sun*(M_star/M_sun) #Saturation Luminosity
pi=math.pi
x0=0.5 #initial water mass fraction



In [153]:
#initalizing variables
t=1e6*yr #1 million years to seconds
Mp=3*M_ea
#Range from 1-20

In [246]:
#functions
#Mp will range from 0-20 in terms of Earth masses

#FOR RADIUS, USE IOP & ERROR IOP
def wmfraction(Mp,Mp0,X0):
    return(1-((Mp0-(X0*Mp0))/Mp))
def Rp_function(CMF,X,Teq,Mp): #Mp in kg, returns Rp in meters
    Mp_e=Mp/M_ea
    Rp_e=iop.radius_iop(CMF,X,Teq,Mp_e)
    return(Rp_e*R_ea)#X is water mass fraction, function not made yet
#High energy luminosity
def L_He_function(M_star,t): #t in Myr
    if t<t_sat:
        return(L_sat)
    else:
        return((L_sat)*((t/t_sat)**(-1-a_0)))
#Efficiency parameter η - how much HE is converted into mass loss
def eff_par(Mp,Rp):
    v_esc=((2*(G)*Mp)/(Rp))**(0.5)
    return(0.1*(((v_esc)/(15))**(-2)))
#Mass-loss rate
def ml_rate(Mp,Rp,a,t): #In earth masses/Billion Years
    L_HiE=L_He_function(M_star,t)
    e_p=eff_par(Mp,Rp)
    return(e_p*((Rp**3)*L_HiE)/(4*(a**2)*(G)*Mp)) #given in kg/s

    

In [244]:
Rp=Rp_function(0.325,0,300,1*M_ea)
print(Rp/R_ea)

0.9949873614486793


In [247]:
#function testing
print(wmfraction(0.75*M_ea,1*M_ea,x0))
print(L_He_function(M_sun,1e9*yr))
print(eff_par(M_ea,R_ea))
print(ml_rate(M_ea,R_ea,a,t)*(1e9*yr)/M_ea)   #Earth masses per Gyr
print(ml_rate(M_ea,R_ea,a,t)) #kg/s

0.33333333333333326
2.5456200000000004e+21
1.7992879756291102e-07
8.809254029841373e-10
0.1668269242277979


In [266]:
#Evolution
t_evo=1e6*yr #time
del_t=1e6*yr #increment of time changing per 1 million years
tmax=5e9*yr #placeholder, to prevent computer overload
Mp_0=4.53 #original mass of planet
X_0=0.5 #original WMF
Rp_evo=1.704*R_ea #radius of the planet that will change over time
Mp_evo=4.53 #mass of the planet that will change over time
X_evo=0.5 #WMF that will change over time

while X_evo>=0: 
    if t_evo>tmax:
        print("time")
        break
    t_evo=t_evo+del_t #increasing time
    M_multi=((ml_rate(Mp_evo*M_ea,Rp_evo,a,t_evo)*del_t)/M_ea) #knowing that Mp1 = Mp0-(Mdot_0*delta_t)
    Mp_evo=(Mp_evo-M_multi)
    X_evo=wmfraction(Mp_evo*M_ea,Mp_0*M_ea,X_0)
    Rp_evo=Rp_function(cmf_p,X_evo,Teq,Mp_evo*M_ea)
    print(Rp_evo/R_ea)
    
        
   

24.23277260616442
2.398553036684894
2.3985530365934684
2.398553036502043
2.3985530364106165
2.398553036319191
2.3985530362277645
2.3985530361363385
2.398553036044913
2.398553035953487
2.3985530358620606
2.398553035770635
2.3985530356792086
2.398553035587783
2.3985530354963576
2.398553035404931
2.398553035313505
2.3985530352220796
2.3985530351306537
2.3985530350392272
2.3985530349478013
2.3985530348563757
2.3985530347649493
2.3985530346735238
2.398553034582098
2.398553034490672
2.398553034399246
2.39855303430782
2.3985530342163943
2.3985530341249683
2.398553034033543
2.3985530339421164
2.39855303385069
2.398553033759265
2.3985530336678385
2.3985530335764125
2.3985530334849865
2.3985530333935605
2.398553033302134
2.3985530332107086
2.398553033119283
2.398553033027857
2.3985530329364306
2.398553032845005
2.398553032753579
2.3985530326621527
2.3985530325707276
2.398553032479301
2.398553032387875
2.3985530322964492
2.3985530322050232
2.3985530321135977
2.3985530320221713
2.3985530319307458


In [55]:
#deltat should be 1 million years

3.141592653589793